## This code illustrates how to get S&P500 index constituents and their identifiers from CRSP and Compustat
- Researchers used to be able to extract index membership information from Compustat's "comp.idxcst_his" data. Now that S&P pulled this piece of data off from WRDS platform, we have to turn to CRSP to get S&P500 Index membership data. 
- Unfortunately, there is no easy way to uncover constituents info for the other indices covered by "comp.idxcst_his".

In [2]:
##########################################
# S&P 500 Index Constituents             #
# Qingyi (Freda) Song Drechsler          #
# Date: October 2020                     #
##########################################

import pandas as pd
import wrds

### Step 1: Connect to WRDS

In [3]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


### Step 2: Get SP500 Index Membership from CRSP
- I opt for the monthly frequency of the data, but one can choose to work with crsp.dsp500list if more precise date range is needed.

In [8]:
sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.dsp500list as a,
                        crsp.dsf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/2000'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])

In [10]:
sp500.head()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\IPython\core\formatters.py", line 223, in catch_format_error
    r = method(self, *args, **kwargs)
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\IPython\core\formatters.py", line 344, in __call__
    return method()
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\pandas\core\frame.py", line 1175, in _repr_html_
    else:
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\pandas\io\formats\format.py", line 1074, in to_html
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\pandas\io\formats\html.py", line 88, in to_string
    lines = self.render()
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\pandas\io\formats\html.py", line 644, in render
    super().render()
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\pandas\io\formats\html.py", line 94, in render
    self._write_

   permno      start     ending       date       ret
0   17005 1957-03-01 2023-12-29 2000-01-03 -0.051724
1   22293 1957-03-01 2023-12-29 2000-01-03 -0.055259
2   14322 1925-12-31 2005-03-24 2000-01-03 -0.010288
3   10145 1925-12-31 2023-12-29 2000-01-03 -0.017335
4   25232 1957-03-01 2001-08-29 2000-01-03 -0.012438

### Step 3: Add Other Company Identifiers from CRSP.MSENAMES
- You don't need this step if only PERMNO is required
- This step aims to add TICKER, SHRCD, EXCHCD and etc. 

In [11]:
# Add Other Descriptive Variables

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt, 
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

In [15]:
# mse.sample(5)
mse

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\IPython\core\formatters.py", line 223, in catch_format_error
    r = method(self, *args, **kwargs)
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\IPython\core\formatters.py", line 344, in __call__
    return method()
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\pandas\core\frame.py", line 1175, in _repr_html_
    else:
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\pandas\io\formats\format.py", line 1074, in to_html
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\pandas\io\formats\html.py", line 88, in to_string
    lines = self.render()
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\pandas\io\formats\html.py", line 644, in render
    super().render()
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\pandas\io\formats\html.py", line 94, in render
    self._write_

                           comnam    ncusip     namedt   nameendt  permno  \
0       OPTIMUM MANUFACTURING INC  68391610 1986-01-07 1986-12-03   10000   
1       OPTIMUM MANUFACTURING INC  68391610 1986-12-04 1987-03-09   10000   
2       OPTIMUM MANUFACTURING INC  68391610 1987-03-10 1987-06-11   10000   
3              GREAT FALLS GAS CO  39040610 1986-01-09 1993-11-21   10001   
4                 ENERGY WEST INC  29274A10 1993-11-22 2004-06-09   10001   
...                           ...       ...        ...        ...     ...   
113851              S & W SEED CO  78513510 2010-06-14 2023-12-29   93434   
113852      SINO CLEAN ENERGY INC  82936G20 2010-06-14 2012-05-18   93435   
113853           TESLA MOTORS INC  88160R10 2010-06-29 2017-02-01   93436   
113854                  TESLA INC  88160R10 2017-02-02 2023-04-02   93436   
113855                  TESLA INC  88160R10 2023-04-03 2023-12-29   93436   

        shrcd  exchcd  hsiccd ticker  
0          10       3    3990  OMFGA

In [14]:
# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]
sp500_full.sample(5)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\IPython\core\formatters.py", line 223, in catch_format_error
    r = method(self, *args, **kwargs)
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\IPython\core\formatters.py", line 344, in __call__
    return method()
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\pandas\core\frame.py", line 1175, in _repr_html_
    else:
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\pandas\io\formats\format.py", line 1074, in to_html
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\pandas\io\formats\html.py", line 88, in to_string
    lines = self.render()
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\pandas\io\formats\html.py", line 644, in render
    super().render()
  File "c:\Users\givan\anaconda3\envs\pytorch-gpu\lib\site-packages\pandas\io\formats\html.py", line 94, in render
    self._write_

          permno      start     ending       date       ret  \
20528186   11896 2018-12-03 2021-08-27 2021-08-18  0.011423   
9476856    52476 1997-06-19 2023-12-29 2009-11-16  0.008932   
16115569   70033 2000-01-31 2020-06-19 2016-11-14  0.000339   
2855600    61735 1996-04-23 2023-12-29 2003-01-09  0.034770   
22272938   89217 2015-07-09 2023-08-24 2023-08-01 -0.009007   

                                 comnam    ncusip     namedt   nameendt  \
20528186  MAXIM INTEGRATED PRODUCTS INC  57772K10 2008-10-08 2021-08-25   
9476856                     EQUIFAX INC  29442910 2004-06-10 2021-03-31   
16115569            HARLEY DAVIDSON INC  41282210 2013-05-30 2023-12-29   
2855600                        AON CORP  03738910 2002-01-02 2004-02-16   
22272938         ADVANCE AUTO PARTS INC  00751Y10 2023-03-24 2023-12-29   

          shrcd  exchcd  hsiccd ticker  
20528186     11       3    3674   MXIM  
9476856      11       1    7389    EFX  
16115569     11       1    3751    HOG  
285560

### Step 4: Add Compustat Identifiers
- Link with Compustat's GVKEY and IID if need to work with fundamental data
- Linkage is done through crsp.ccmxpf_linktable

In [8]:
# Linking with Compustat through CCM

ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

In [15]:
# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]
sp500ccm.sample(5)

,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker,gvkey,iid,linktype,linkprim,linkdt,linkenddt
42113,48274.0,1989-08-03,2011-12-12,2005-05-31,0.068434,NICOR INC,65408610,2004-06-10,2011-12-09,11.0,1.0,4924.0,GAS,007904,01,LC,P,1969-05-02,2011-12-30 00:00:00.000000
21022,27887.0,1972-09-14,2019-12-31,2002-08-30,-0.090704,BAXTER INTERNATIONAL INC,07181310,2002-02-01,2004-02-16,11.0,1.0,3841.0,BAX,002086,01,LC,P,1962-01-31,2020-10-21 14:26:10.563769
81974,90829.0,2008-08-27,2019-12-31,2010-08-31,0.140535,C F INDUSTRIES HOLDINGS INC,12526910,2005-08-11,2019-12-31,11.0,1.0,2873.0,CF,163946,01,LC,P,2005-08-11,2020-10-21 14:26:10.563769
39158,13856.0,1957-03-01,2019-12-31,2004-12-31,0.050491,PEPSICO INC,71344810,2004-06-10,2017-12-19,11.0,1.0,2086.0,PEP,008479,01,LC,P,1962-01-31,2020-10-21 14:26:10.563769
137326,71175.0,1994-03-11,2019-12-31,2018-02-28,-0.041925,UNUM GROUP,91529Y10,2007-03-02,2019-09-11,11.0,1.0,6321.0,UNM,012726,01,LU,P,1986-12-31,2020-10-21 14:26:10.563769


In [16]:
# Rearrange columns for final output

sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', \
                                  'linktype', 'linkprim', 'linkdt', 'linkenddt'])
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip', 'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                     'gvkey', 'iid', 'start', 'ending', 'ret']]
sp500ccm.sample(5)

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
49267,2006-04-28,86102.0,FEDERATED INVESTORS INC PA,31421110,11.0,1.0,6282.0,FII,066731,01,2003-04-16,2012-12-31,-0.101152
120416,2015-11-30,81910.0,INVESCO LTD,G491BT10,12.0,1.0,6282.0,IVZ,029804,02,2008-08-21,2019-12-31,0.023817
112968,2014-11-28,78975.0,INTUIT INC,46120210,11.0,3.0,7370.0,INTU,027928,01,2000-12-11,2019-12-31,0.066583
19214,2002-05-31,64186.0,C I G N A CORP,12550910,11.0,1.0,6324.0,CI,002547,01,1982-04-08,2019-12-31,-0.027064
134250,2017-09-29,15707.0,HEWLETT PACKARD ENTERPRISE CO,42824C10,11.0,1.0,7374.0,HPE,026156,01,2015-11-02,2019-12-31,0.046962


In [21]:
cnt = sp500ccm.groupby(['date'])['permno'].count().reset_index().rename(columns={'permno':'npermno'})
cnt.sample(4)

,date,npermno
44,2003-09-30,500
212,2017-09-29,501
84,2007-01-31,500
39,2003-04-30,500


In [22]:
sp500ccm.sample(10)

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
113348,2014-11-28,26825.0,KELLOGG CO,48783610,11.0,1.0,2043.0,K,006375,01,1961-04-26,2019-12-31,0.043465
507,2000-01-31,80599.0,LEHMAN BROTHERS HOLDINGS INC,52490810,11.0,1.0,6282.0,LEH,030128,01,1998-01-12,2008-09-16,-0.157934
60387,2007-10-31,40272.0,INTERNATIONAL FLAVORS & FRAG INC,45950610,11.0,1.0,2869.0,IFF,006078,01,1976-03-04,2019-12-31,-0.012297
50310,2006-06-30,78034.0,PATTERSON COMPANIES INC,70339510,11.0,3.0,5990.0,PDCO,025880,01,2005-10-11,2018-03-16,0.019854
97932,2012-10-31,16600.0,HERSHEY CO,42786610,11.0,1.0,2066.0,HSY,005597,01,1957-03-01,2019-12-31,-0.028777
70840,2009-02-27,24766.0,NORTHROP GRUMMAN CORP,66680710,11.0,1.0,3812.0,NOC,007985,01,1985-06-13,2019-12-31,-0.215295
23639,2002-12-31,13119.0,MAYTAG CORP,57859210,11.0,1.0,3633.0,MYG,007139,01,1960-12-14,2006-03-31,-0.079457
112012,2014-09-30,28345.0,MURPHY OIL CORP,62671710,11.0,1.0,1382.0,MUR,007620,01,2005-08-15,2017-07-25,-0.089003
93624,2012-03-30,52329.0,JACOBS ENGINEERING GROUP INC,46981410,11.0,1.0,8711.0,JEC,006216,01,2007-10-26,2019-12-31,-0.040026
114295,2015-01-30,91461.0,WESTERN UNION CO,95980210,11.0,1.0,6099.0,WU,175263,01,2006-10-02,2019-12-31,-0.050810
